# Notebook to test model3

In [ ]:
# load json and create model
import os
from PIL import Image
import matplotlib.pyplot as plt
from skimage.color import rgb2lab, lab2rgb, rgb2gray
from skimage.io import imsave
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from tensorflow.keras.layers import Dense, Dropout, Conv2D, Flatten, Input, UpSampling2D
from tensorflow.keras.layers import Reshape, Conv2DTranspose, MaxPooling2D, MaxPool2D
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.utils import plot_model
from tensorflow.keras.backend import int_shape
from tensorflow.keras.models import load_model
from tensorflow.keras.models import model_from_json


## Set up paths and current working directory, PLEASE change to yours!!!!

In [ ]:
os.chdir('C:/Users/YOUR_USER/Documents/Colourising_Photos')
# Select a path to read input
inputPath = os.getcwd()+"/example_inputs/p_512/";
outputPath = os.getcwd()+"/test_output/Model3/";

### Loading from model.json the model and then loading in weights from model.h5

In [ ]:

json_file = open(''+os.getcwd()+'/Model3/model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()

model = Sequential()

model.add(Conv2D(64, (3, 3), input_shape=(512, 512, 1), activation='relu', padding='same'))

model.add(Conv2D(64, (3, 3), activation='relu', padding='same', strides=2))
model.add(Conv2D(128, (3, 3), activation='relu', padding='same', strides=2))
model.add(Conv2D(256, (3, 3), activation='relu', padding='same', strides=2))
model.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(256, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(2, (3, 3), activation='tanh', padding='same'))
model.add(UpSampling2D((2, 2)))

# load weights into new model
model.load_weights(""+os.getcwd()+"/Model3/model.h5")
model.compile('RMSprop','mean_squared_error')


## Select folder of 512x512 images to colourise

In [ ]:
colourise = []

for filename in os.listdir(inputPath):
    colourise.append(img_to_array(load_img(inputPath+filename)))

colourise = rgb2lab(1.0/255*(np.array(colourise, dtype=float)))[:,:,:,0]
colourise = colourise.reshape(colourise.shape+(1,))


### Now colourise will be predicted and sent to /test_outputs/Model3

In [ ]:

output = model.predict(colourise)
output = output * 128
i = 0;
for i in range(0,len(output)):
    temp = np.zeros((512,512,3))
    temp[:,:,0] = colourise[i][:,:,0];
    temp[:,:,1:] = output[i];
    temp_output = lab2rgb(temp)
    temp_output *= 255
    temp_output = temp_output.astype('uint8')
    image = Image.fromarray( temp_output )
    image.save(outputPath+'ouput{}.png'.format(i))